# **Titanic**

The objective of this notebook is to predict whether a passenger onboard the titanic will survive given features such as their gender, age, class etc.

## 1. Preparations

We will begin by installing all of the necessary libraries and loading in the data

### 1.1. Dependencies

In [1]:
!pip install kaggle 
!pip install scikit-learn

In [3]:
import kaggle 
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

api.competition_download_file('titanic', 'train.csv')
api.competition_download_file('titanic', 'test.csv')

import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

100%|██████████| 59.8k/59.8k [00:00<00:00, 3.90MB/s]


100%|██████████| 28.0k/28.0k [00:00<00:00, 28.2MB/s]

### 1.2. Importing Data

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## 2. Preprocessing

We can now begin cleaning the data so all of the relevant features are included and formatted in a suitable way to run a machine learning model.

### 2.1. Removing Extraneous Features

In [7]:
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)
train.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


### 2.2. Remove Missing Data

In [9]:
print(train.shape)
train.isnull().sum()

(891, 8)


Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

As can be seen, there are only 2 passengers with an unknown departure point but the ages of 177 passengers are also missing. Since this is ~ 20% of the full dataset, missing value imputation needs to be conducted. But first, I will remove the two passengers with missing embarkment locations.

In [10]:
train.dropna(subset = ['Embarked'], inplace = True)
train.shape

(889, 8)

### 2.3. One-Hot Encoding

In [ ]:
#testing

In [ ]:
mice_imputer = IterativeImputer()
train = mice_imputer.fit_transform(train)